# 3.2. Querying an encrypted model
**protecting privacy and IP simultaneously**


### Set up

In [1]:
# Import dependencies
import grid as gr
from grid import syft as sy
import torch as th
import skin_cancer_model_utils as scmu

# Hook torch
hook = sy.TorchHook(th)
me = hook.local_worker
me.is_client_worker = False
    
# Connect to nodes
grid_server = gr.WebsocketGridClient(hook, "http://localhost:3001", id="alice")
patient_server = gr.WebsocketGridClient(hook, "http://localhost:3000", id="bob")
hospital_server = gr.WebsocketGridClient(hook, "http://localhost:3002", id="james")
crypto_provider = gr.WebsocketGridClient(hook, "http://localhost:3003", id="dan")

grid_server.connect()
patient_server.connect()
hospital_server.connect()
crypto_provider.connect()

# Connect nodes to each other
gr.connect_all_nodes([grid_server, patient_server, hospital_server, crypto_provider])

### Query model
#### Get a copy of the encrypted model

In [2]:
encrypted_model = grid_server.download_model("convnet")

**Encrypted data for running through the model**

In [6]:
df = scmu.read_skin_cancer_dataset()
train_df, valid_df, test_df = scmu.split_data(df)

params = {'batch_size': 1,
          'shuffle': True,
          'num_workers': 6}

# These values are from 3.1.
input_size = 32
train_mean = train_mean, train_std = (th.tensor([0.6979, 0.5445, 0.5735]), th.tensor([0.0959, 0.1187, 0.1365]))

test_set = scmu.Dataset(test_df, transform=scmu.transform(input_size, train_mean, train_std))
test_generator = th.utils.data.DataLoader(test_set, **params)

data, target = next(iter(test_generator))

/home/marianne/PySyft/syft/frameworks/torch/hook/hook.py:483: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  current_tensor = hook_self.torch.native_tensor(*args, **kwargs)
/home/marianne/PySyft/syft/frameworks/torch/hook/hook.py:483: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  current_tensor = hook_self.torch.native_tensor(*args, **kwargs)
/home/marianne/PySyft/syft/frameworks/torch/hook/hook.py:483: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  current_tensor = hook_self.torch.native_tensor(*args, **kwargs)
/home/marianne/PySyft/syft/f

In [7]:
x_sh = data.encrypt(patient_server, hospital_server, crypto_provider=crypto_provider)

#### Run encrypted inference

In [8]:
%%time
print(encrypted_model(x_sh).request_decryption())

tensor([[-0.0820,  0.2490]])
CPU times: user 5min 27s, sys: 2min 10s, total: 7min 37s
Wall time: 7min 29s


# Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

## Star PySyft on GitHub
The easiest way to help our community is just by starring the GitHub repos! This helps raise awareness of the cool tools we're building.

[Star PySyft](https://github.com/OpenMined/PySyft)

## Join our Slack!
The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at http://slack.openmined.org

## Join a Code Project!
The best way to contribute to our community is to become a code contributor! At any time you can go to PySyft GitHub Issues page and filter for "Projects". This will show you all the top level Tickets giving an overview of what projects you can join! If you don't want to join a project, but you would like to do a bit of coding, you can also look for more "one off" mini-projects by searching for GitHub issues marked "good first issue".

[PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
[Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

## Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)